In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed,LSTM
import numpy as np
import pandas as pd
import copy
df = pd.read_csv("../Data/data_6_nodes_yen.csv") #You need to change #directory accordingly
df.head()

X(1, 2)  X(1, 4)  X(1, 5)  X(2, 3)  X(2, 6)  X(3, 6)  X(4, 5)  X(5, 6)  \
0       90       80       40       40       10       20       20       40   
1       40       20       80       80       30       70       80       20   
2       70       20       80       30       60       70       80       90   
3       80       40       50       30       30       80       90       60   
4       20       30       30       20       60       70       20       10   

   Y1(1, 2)  Y1(1, 4)  ...  Y3(4, 5)  Y3(5, 6)  Y4(1, 2)  Y4(1, 4)  Y4(1, 5)  \
0         0         0  ...         1         1         1         0         0   
1         1         0  ...         1         1         1         0         0   
2         1         0  ...         0         0         0         1         0   
3         0         0  ...         1         1         1         0         0   
4         0         0  ...         0         0         1         0         0   

   Y4(2, 3)  Y4(2, 6)  Y4(3, 6)  Y4(4, 5)  Y4(5, 6)  
0         1         0         1         0         0  
1         1         0         1         0         0  
2         0         0         0         1         1  
3         1         0         1         0         0  
4         1         0         1         0         0  

[5 rows x 40 columns]

In [3]:
edges = [[1, 2], [1, 4], [1, 5], [2, 3], [2, 6], [3, 6], [4, 5], [5, 6]]

In [4]:
number_edges = 8
number_steps = 3
start_node = 1
end_node = 6

In [5]:
X = df.iloc[:30000,:number_edges].values
y = df.iloc[:30000,number_edges:(number_edges*(number_steps+1))].values

In [6]:
X.shape

(30000, 8)

In [7]:
y.shape

(30000, 24)

In [8]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [9]:
X2[1]

array([80, 50, 40, 80, 60, 40, 30, 60], dtype=int64)

In [10]:
y_test[1]

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1], dtype=int64)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [12]:
X_train = np.repeat(X_train[:, np.newaxis, : ], number_steps, axis=1)
X_train.shape

(27000, 3, 8)

In [13]:
X_test = np.repeat(X_test[:, np.newaxis, : ], number_steps, axis=1)
X_test.shape

(3000, 3, 8)

In [14]:
y_train[0]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0], dtype=int64)

In [15]:
y_train.shape

(27000, 24)

In [16]:
y_train_2 = np.reshape(y_train,(y_train.shape[0], number_steps, number_edges))
y_train_2.shape

(27000, 3, 8)

In [17]:
y_train_2[0]

array([[0, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 1, 0, 0, 0]], dtype=int64)

In [18]:
y_test_2 = np.reshape(y_test,(y_test.shape[0], number_steps, number_edges))
y_test_2.shape

(3000, 3, 8)

In [19]:
X2[1,:]

array([80, 50, 40, 80, 60, 40, 30, 60], dtype=int64)

In [20]:
import copy
def cost_path(x,y):
    cost = 0
    path = []
    for i in range(number_edges):
        if y[i]==1:
            path.append(edges[i])
            cost+= x[i]
    if(len(path)!=0):
        path_2 = copy.deepcopy(path)
        path_nodes = [start_node]
        if(path[0][0]!=start_node):
            cost = 0
        else:
            path_nodes.append(path[0][1])
            del path_2[0]
            condition = True
            while(len(path_2)!=0 and condition):
                condition = False
                for i in range(len(path_2)):
                    if(path_nodes[-1] in path_2[i]):
                        path_2[i].remove(path_nodes[-1])
                        path_nodes.append(path_2[i][0])
                        path_2.pop(i)
                        condition = True
                        break
        if(path_nodes[-1]!=end_node or len(path_2)!=0):
            cost = 0
    else:
        cost = 0
    return cost

In [22]:
class Network:
    def __init__(self):
        self.model = Sequential()
        self.model.add(LSTM(16, input_shape=(number_steps, number_edges), return_sequences=True))
        self.model.add(TimeDistributed(Dense(number_edges, activation='sigmoid')))
        self.model.compile(loss='BinaryCrossentropy', optimizer='sgd')
        self.fitness = float("inf")
  
    def set_weights(self, weights):
        self.model.set_weights(weights) 
    
    def get_weights(self):
        return self.model.get_weights()
    
    def architecture(self):
        self.model.summary()
    
    def train(self):
        self.model.fit(X_train, y_train_2, epochs=100, batch_size=64)
    
    def evaluate(self):
        loss = self.model.evaluate(X_train, y_train_2)
        return loss
    
    def get_accuracy(self):
        pred = self.model.predict(X_train)
        accuracy = np.zeros(number_steps+1)
        for i in range(pred.shape[0]):
            condition_1 = True
            for t in range(number_steps):
                output = copy.deepcopy(pred[i][t])
                condition_2 = True
                for j in range(len(output)):
                    output[j] = round(output[j])
                if(cost_path(X1[i],output) != cost_path(X1[i],y_train_2[i][t])):
                    condition_2 = False
                    condition_1 = False
                if condition_2:
                    accuracy[t] += 1
            if condition_1:
                accuracy[number_steps] += 1
        accuracy = np.round(accuracy / pred.shape[0] * 100,2)
        return accuracy[-1]

In [23]:
class GA:
    def __init__(self, N, Max, Pc, Pm, Ts):
        self.N = N
        self.Max = Max
        self.Pm = Pm
        self.Pc = Pc
        self.population = [Network() for i in range(self.N)]
        self.best = Network()
        self.Ts = Ts
        self.children_population_weights = []
        self.shapes = [a.shape for a in Network().get_weights()]
    
    def Train_generation(self):
        i =  0
        for member in self.population:
            print("Train for member: ",i)
            i+=1
            member.train()
    
    def Train_best(self):
        self.best.train()
    
    def Flatten(self, W):
        flattened = np.concatenate([a.flatten() for a in W])
        return flattened
    
    def Un_Flatten(self, flattened):
        newarray = []
        index = 0
        for shape in self.shapes:
            size = np.product(shape)
            newarray.append(flattened[index : index + size].reshape(shape))
            index += size
        return newarray

    def Crossover(self):
        self.children_population_weights.clear()
        for i in range(0,self.N,2):
            father = np.random.randint(len(self.population))
            mother = np.random.randint(len(self.population))
            father_w0 = self.population[father].get_weights()
            mother_w0 = self.population[mother].get_weights()
            if np.random.uniform(0,1) < self.Pc:
                father_weights = self.Flatten(father_w0)
                mother_weights = self.Flatten(mother_w0)
                k = np.random.randint(0,int(len(father_weights)/4))
                for ii in range(k):
                    mid_point = np.random.choice(list(range(len(father_weights))))
                    temp = father_weights[mid_point]
                    father_weights[mid_point] = mother_weights[mid_point]
                    mother_weights[mid_point] = temp
                father_w = self.Un_Flatten(father_weights)
                mother_w = self.Un_Flatten(mother_weights)
                self.children_population_weights.append(father_w)
                self.children_population_weights.append(mother_w)
            else:
                self.children_population_weights.append(father_w0)
                self.children_population_weights.append(mother_w0)
        self.population = [Network() for i in range(len(self.children_population_weights))]
        for i in range(len(self.children_population_weights)):
            self.population[i].set_weights(self.children_population_weights[i])
        
    def Mutation(self):
        for member in self.population:
            if np.random.uniform(0,1) < self.Pm:
                old_weight = self.Flatten(member.get_weights())
                k = np.random.randint(0,int(len(old_weight)/4))
                for ii in range(k):
                    mid_point = np.random.choice(list(range(len(old_weight))))
                    old_weight[mid_point] = old_weight[mid_point] + np.random.uniform(-0.1,0.1)
                new_weight = self.Un_Flatten(old_weight)
                member.set_weights(new_weight)
    
    def Calculate_fitness(self):
        for i in range(len(self.population)):
            self.population[i].fitness = self.population[i].evaluate()
    
    def Selection(self):
        self.population.sort(key=lambda x: x.fitness)
        del self.population[int(self.Ts*len(self.population)):]
    
    def GetBest(self):
        if self.population[0].fitness < self.best.fitness:
            self.best.set_weights(self.population[0].get_weights())
            self.best.fitness = self.population[0].fitness

    def Do(self):
        for i in range(self.Max):
            print("i = ",i)
            self.Crossover()
            self.Mutation()
            self.Calculate_fitness()
            self.Selection()
            self.GetBest()
            print("best fitness:",self.best.fitness)

In [26]:
N = 10
Max = 1000
Pc = 0.7
Pm = 0.5
Ts = 0.6

In [27]:
ga = GA(N, Max, Pc, Pm, Ts)
ga.Do()

i =  0
844/844 [==============================] - 2s 2ms/step - loss: 0.7002
best fitness: 0.6953088641166687
i =  1
844/844 [==============================] - 2s 2ms/step - loss: 0.6989
best fitness: 0.6951332688331604
i =  2
844/844 [==============================] - 2s 2ms/step - loss: 0.6962
best fitness: 0.6928724646568298
i =  3
844/844 [==============================] - 2s 2ms/step - loss: 0.6962
best fitness: 0.6928724646568298
i =  4
844/844 [==============================] - 2s 2ms/step - loss: 0.6956
best fitness: 0.6927458643913269
i =  5
844/844 [==============================] - 2s 2ms/step - loss: 0.6969
best fitness: 0.6925795078277588
i =  6
844/844 [==============================] - 2s 2ms/step - loss: 0.6939
best fitness: 0.6925795078277588
i =  7
844/844 [==============================] - 2s 2ms/step - loss: 0.6908
best fitness: 0.6908197999000549
i =  8
844/844 [==============================] - 2s 2ms/step - loss: 0.6911
best fitness: 0.6908116936683655
i =  9
844

844/844 [==============================] - 2s 2ms/step - loss: 0.6928
best fitness: 0.6848993301391602
i =  12
844/844 [==============================] - 2s 2ms/step - loss: 0.6921
best fitness: 0.6848043203353882
i =  13
844/844 [==============================] - 2s 2ms/step - loss: 0.6891
best fitness: 0.6814658045768738
i =  14
844/844 [==============================] - 2s 2ms/step - loss: 0.6852
best fitness: 0.6814658045768738
i =  15
844/844 [==============================] - 2s 2ms/step - loss: 0.6815
best fitness: 0.6787533760070801
i =  16
844/844 [==============================] - 2s 2ms/step - loss: 0.6828
best fitness: 0.6787533760070801
i =  17
844/844 [==============================] - 2s 2ms/step - loss: 0.6810
best fitness: 0.6758238077163696
i =  18
844/844 [==============================] - 2s 2ms/step - loss: 0.6839
best fitness: 0.6758238077163696
i =  19
844/844 [==============================] - 2s 2ms/step - loss: 0.6784
best fitness: 0.6758238077163696
i =  20
8

844/844 [==============================] - 2s 2ms/step - loss: 0.6779
best fitness: 0.67156583070755
i =  23
844/844 [==============================] - 2s 2ms/step - loss: 0.6759
best fitness: 0.6713153123855591
i =  24
844/844 [==============================] - 2s 2ms/step - loss: 0.6732
best fitness: 0.6713153123855591
i =  25
844/844 [==============================] - 2s 2ms/step - loss: 0.6757
best fitness: 0.6713153123855591
i =  26
844/844 [==============================] - 2s 2ms/step - loss: 0.6763
best fitness: 0.6713153123855591
i =  27
844/844 [==============================] - 2s 2ms/step - loss: 0.6729
best fitness: 0.6713153123855591
i =  28
844/844 [==============================] - 2s 2ms/step - loss: 0.6730
best fitness: 0.6713153123855591
i =  29
844/844 [==============================] - 1s 1ms/step - loss: 0.6726
best fitness: 0.6710132360458374
i =  30
844/844 [==============================] - 1s 1ms/step - loss: 0.6727
best fitness: 0.6710132360458374
i =  31
844

844/844 [==============================] - 2s 1ms/step - loss: 0.6717
best fitness: 0.6707510352134705
i =  34
844/844 [==============================] - 2s 2ms/step - loss: 0.6708
best fitness: 0.6692478060722351
i =  35
844/844 [==============================] - 2s 2ms/step - loss: 0.6711
best fitness: 0.6688271164894104
i =  36
844/844 [==============================] - 2s 2ms/step - loss: 0.6709
best fitness: 0.6675670146942139
i =  37
844/844 [==============================] - 2s 2ms/step - loss: 0.6696
best fitness: 0.6675670146942139
i =  38
844/844 [==============================] - 2s 2ms/step - loss: 0.6679
best fitness: 0.6675670146942139
i =  39
844/844 [==============================] - 2s 2ms/step - loss: 0.6687
best fitness: 0.666474461555481
i =  40
844/844 [==============================] - 2s 2ms/step - loss: 0.6665
best fitness: 0.6662983298301697
i =  41
844/844 [==============================] - 2s 2ms/step - loss: 0.6682
best fitness: 0.6632739901542664
i =  42
84

844/844 [==============================] - 2s 1ms/step - loss: 0.6631
best fitness: 0.6601060628890991
i =  45
844/844 [==============================] - 2s 1ms/step - loss: 0.6622
best fitness: 0.6584319472312927
i =  46
844/844 [==============================] - 2s 2ms/step - loss: 0.6622
best fitness: 0.6572657227516174
i =  47
844/844 [==============================] - 2s 1ms/step - loss: 0.6618
best fitness: 0.6572657227516174
i =  48
844/844 [==============================] - 2s 2ms/step - loss: 0.6601
best fitness: 0.6560601592063904
i =  49
844/844 [==============================] - 2s 2ms/step - loss: 0.6586
best fitness: 0.6560601592063904
i =  50
844/844 [==============================] - 2s 2ms/step - loss: 0.6606
best fitness: 0.6560601592063904
i =  51
844/844 [==============================] - 2s 2ms/step - loss: 0.6568
best fitness: 0.6560601592063904
i =  52
844/844 [==============================] - 2s 2ms/step - loss: 0.6568
best fitness: 0.6543551683425903
i =  53
8

...

844/844 [==============================] - 2s 1ms/step - loss: 0.6144
best fitness: 0.6125615835189819
i =  111
844/844 [==============================] - 2s 1ms/step - loss: 0.6158
best fitness: 0.6120479702949524
i =  112
844/844 [==============================] - 2s 1ms/step - loss: 0.6125
best fitness: 0.6113489866256714
i =  113
844/844 [==============================] - 2s 1ms/step - loss: 0.6175
best fitness: 0.6113489866256714
i =  114
844/844 [==============================] - 2s 1ms/step - loss: 0.6115
best fitness: 0.6089757084846497
i =  115
844/844 [==============================] - 2s 1ms/step - loss: 0.6128
best fitness: 0.6089757084846497
i =  116
844/844 [==============================] - 2s 2ms/step - loss: 0.6124
best fitness: 0.6083649396896362
i =  117
844/844 [==============================] - 2s 2ms/step - loss: 0.6104
best fitness: 0.6079233884811401
i =  118
844/844 [==============================] - 2s 2ms/step - loss: 0.6092
best fitness: 0.6079170107841492
i

844/844 [==============================] - 2s 2ms/step - loss: 0.6072
best fitness: 0.6026262044906616
i =  122
844/844 [==============================] - 2s 1ms/step - loss: 0.6074
best fitness: 0.6026262044906616
i =  123
844/844 [==============================] - 2s 1ms/step - loss: 0.6064
best fitness: 0.6026262044906616
i =  124
844/844 [==============================] - 2s 1ms/step - loss: 0.6060
best fitness: 0.6026262044906616
i =  125
844/844 [==============================] - 2s 1ms/step - loss: 0.6053
best fitness: 0.6026262044906616
i =  126
844/844 [==============================] - 2s 1ms/step - loss: 0.6051
best fitness: 0.6026262044906616
i =  127
844/844 [==============================] - 2s 1ms/step - loss: 0.6067
best fitness: 0.6026262044906616
i =  128
844/844 [==============================] - 2s 1ms/step - loss: 0.6028
best fitness: 0.6026262044906616
i =  129
844/844 [==============================] - 2s 2ms/step - loss: 0.6063
best fitness: 0.6025372743606567
i

844/844 [==============================] - 2s 1ms/step - loss: 0.6028
best fitness: 0.5996531248092651
i =  133
844/844 [==============================] - 2s 1ms/step - loss: 0.6032
best fitness: 0.5993440747261047
i =  134
844/844 [==============================] - 2s 1ms/step - loss: 0.6011
best fitness: 0.5983588099479675
i =  135
844/844 [==============================] - 2s 2ms/step - loss: 0.6001
best fitness: 0.5983460545539856
i =  136
844/844 [==============================] - 2s 1ms/step - loss: 0.5983
best fitness: 0.5983460545539856
i =  137
844/844 [==============================] - 2s 1ms/step - loss: 0.6001
best fitness: 0.5983101725578308
i =  138
844/844 [==============================] - 2s 1ms/step - loss: 0.5983
best fitness: 0.5972513556480408
i =  139
844/844 [==============================] - 2s 2ms/step - loss: 0.5987
best fitness: 0.5972513556480408
i =  140
844/844 [==============================] - 2s 1ms/step - loss: 0.5973
best fitness: 0.5972513556480408
i

844/844 [==============================] - 2s 2ms/step - loss: 0.5962
best fitness: 0.5961576700210571
i =  144
844/844 [==============================] - 2s 1ms/step - loss: 0.5938
best fitness: 0.593754768371582
i =  145
844/844 [==============================] - 2s 2ms/step - loss: 0.5975
best fitness: 0.593754768371582
i =  146
844/844 [==============================] - 2s 2ms/step - loss: 0.5964
best fitness: 0.593754768371582
i =  147
844/844 [==============================] - 2s 1ms/step - loss: 0.5931
best fitness: 0.5931287407875061
i =  148
 26/844 [..............................] - ETA: 1s - loss: 0.5979  

KeyboardInterrupt: 

In [ ]:
ga.best.train()

In [ ]:
ga.best.get_accuracy()

In [ ]:
ga.best.evaluate()